In [1]:
import os

In [5]:
os.getcwd()

'/Volumes/JK/Swizzl/swizzl/services'

In [ ]:
from bs4 import BeautifulSoup
import requests 
from mlearning import sentiment as st
from mlearning import prof
import re
import html

In [9]:
def linkText(link):
    url = "{}".format(link)
    r = requests.get(url) 
    soup = BeautifulSoup(r.content,"lxml") 
    article_text = ''
    mylink = soup.find('div').findAll('p')
    for i in mylink:
        article_text += ''.join(i.findAll(text = True))
    return article_text

In [10]:
def YahooFetch():
    url = "https://news.yahoo.com/rss/"
    url = requests.get(url)
    soup = BeautifulSoup(url.text,'xml')
    
    try:
        # Fetch All Required items
        titles = soup.findAll('title')
        links = soup.findAll('link')
        pubDates = soup.findAll('pubDate')

    except Exception as e :
        
        # Return Empty if titles, links, descriptions, pubdates not found
        print(e)
        return "Error"

    # We don't want the first elements as these are just metadata
    
    titles.pop(0)
    titles.pop(0)
    links.pop(0)
    links.pop(0)
    pubDates.pop(0)

    #Dictionary to hold crawled information

    FeedDict = {}
    temp = []
    id = []
    
    try:
        # Add titles to the Dictionary
        for i in titles:
            
        
        for i in titles:
            temp.append(html.unescape(i.get_text()))
            FeedDict['title'] = temp
            
        # Add link and details regarding text contetn @ link to the Dictionary
        
        temp = []
        temptext = []
        tbscore = []
        vadscore = []
        profvalue = []
        for i in links:
            
            # Append Links
            temp.append(html.unescape(i.get_text()))
            
            # Append Text Content from Links
            textval = linkText(html.unescape(i.get_text()))
            textval = re.sub('\"','\\"',textval)
            textval = " \" " + textval + " \" "
            temptext.append(textval)
            
            # Find Subjectivity, Objectivity of text content
            score = st.sentimentTB(textval)
            tbscore.append(score)
            
            # Find Sentiment of text content
            score = st.sentimentVader(textval)
            vadscore.append(score)
            
            # Find Profanity Score of content
            textval = [textval]
            score = float(prof.predProf(textval))
            profvalue.append(score)
            
            # Add to Feeds Dictionary 
            FeedDict['link'] = temp
            FeedDict['linktext'] = temptext
            FeedDict['tbScore'] = tbscore
            FeedDict['vaderScore'] = vadscore
            FeedDict['prof'] = profvalue
            
        # Add Published Dates to the Dictionary
        temp = []
        for i in pubDates:
            temp.append(i.get_text())
            FeedDict['pubdate'] = temp
        
        print("Success")
    except Exception as e :
        print(e)
        return "Error"
    
    return FeedDict